In [1]:
#Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

covid_path = "Resources/COVID_19.csv"
GDP_path = "Resources/Global_GDP.csv"
cities_path = "Resources/worldcities.csv"

COVID_data = pd.read_csv(covid_path)
GDP_data = pd.read_csv(GDP_path)
cities_data = pd.read_csv(cities_path)

In [2]:
gdp_2019 = GDP_data[['Country Name', 'Country Code', '2019']]
gdp_2019.set_index('Country Name')

,Country Code,2019
Country Name,,
Aruba,ABW,NaN
Afghanistan,AFG,1.910135e+10
Angola,AGO,9.463542e+10
Albania,ALB,1.527808e+10
Andorra,AND,3.154058e+09
...,...,...
Kosovo,XKX,7.926108e+09
"Yemen, Rep.",YEM,NaN
South Africa,ZAF,3.514320e+11


In [3]:
latest_covid = [[]]